# Optimal growth model with Value function iteration and interpolation

This code doesn't work.

In [222]:
using Interpolations
using OptimS
using Plots
plotlyjs()

Plots.PlotlyJSBackend()

In [251]:
function VFI()
    # primitive parameter
    const beta = 0.95 #subjective discount factor
    const sigma = 2.0 # relative risk aversion
    const delta = 0.1 #depriciation rate
    const alpha = 0.33 # capital share

    # Steady state
    aterm = 1.0/beta -(1.0 -delta)
    kstar = alpha/aterm
    kstar = kstar^(1.0/(1.0-alpha))
    # discretize asset space
    amin = 0.1*kstar
    amax = 2.0*kstar
    na   = 250
    agrid = linspace(amin,amax,na)
   
    #initialize value function and so on
    # utility function
    u(c) = (c^(1-sigma))/(1-sigma)

    v0 = u.(agrid)# initial guess of value function
    Tv = zeros(na) # update value function
    pol_a =zeros(na) # policy function
    res = zeros(na)
        
    
    #iteration 
    iter = 0
    const maxiter=300
    const tol =1e-5
    error = 10.0

    # value function iteration
    while   maxiter > iter  && error > tol
        itp = scale(interpolate(v0, BSpline(Cubic(Line())), OnGrid()),agrid)
    
        for (i,asset) in enumerate(agrid)
            objective(a_next) = -u(asset^alpha +(1-delta)*asset- a_next) - beta*itp[a_next]
            res = optimize(objective, 0, asset^alpha+(1-delta)*asset,GoldenSection())
            pol_a[i] = res.minimizer        
            Tv[i] = - res.minimum  
        end
       
        error = maximum(abs.(Tv-v0)) # update error
        v0    = copy(Tv) # update value function
        Tv    = zeros(na)
        iter += 1
    end
    
    #println("converged!")
    #println(iteration)
    return(iter,agrid,v0,pol_a)
end

VFI (generic function with 1 method)

In [252]:
@time iter,agrid,v0,pol_a =VFI()

  1.157605 seconds (36.57 M allocations: 591.102 MiB, 6.80% gc time)


(224, 0.31608601990722374:0.02411901356721787:6.321720398144474, [-22.9326, -22.7669, -22.6123, -22.4672, -22.3306, -22.2015, -22.0791, -21.9626, -21.8516, -21.7455  …  -15.7622, -15.752, -15.7417, -15.7316, -15.7214, -15.7113, -15.7013, -15.6913, -15.6813, -15.6713], [0.492229, 0.518677, 0.544816, 0.570681, 0.5963, 0.621698, 0.646895, 0.671908, 0.696755, 0.721447  …  5.77991, 5.80124, 5.82258, 5.84391, 5.86524, 5.88657, 5.90789, 5.92922, 5.95055, 5.97187])

In [253]:
plot(agrid,pol_a, label="policy function",color="blue",linewidth=1.5)
plot!(agrid,agrid, label="45 degree",color="red",linewidth=1.5)


In [254]:
plot(agrid,v0,label="value function",color="blue",linewidth=1.5)